Телеграм-бот – помощник по созданию персонажа для настольной игры Dungeons & Dragons

Импортируем необходимые библиотеки:

In [10]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import csv
import re
import telebot
import csv
from telebot import types

Парсим сайт dnd.su — основной русскоязычный ресурс со всей необходимой нам информацией, из которой впоследствии мы создадим DataFrame и csv-файл:

In [15]:
# Главная сложность состояла в том, что система правил в D&D — это лишь рекомендации
# Поэтому существует огромное количество "неофициально" созданных рас, классов, предысторий и т. д.
# Для бота я решила использовать только классические расы и предыстории из "Книги игрока" (Player's Handbook)
# Поэтому сначала парсила с днд.су и сокращала по возможности из неофициальных источников то, что могу сократить, а остальное решила дочистить в датасете 

url_races = 'https://dnd.su/race/'
url_classes = 'https://dnd.su/class/'
url_backgrounds = 'https://dnd.su/backgrounds/'

def parse_background_skills(url):
    time.sleep(1)  
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    skills_list = ''
    tools_list = ''
    equipment_list = ''
    
 
    description_section = soup.find(itemprop="description")
    if description_section:
        skills_section = description_section.find('strong', text=re.compile(r'Владение навыками:'))
        if skills_section:
            skills_paragraph = skills_section.find_parent('p')
            if skills_paragraph:
                skills_spans = skills_paragraph.find_all('span')
                skills_list = ', '.join(span.get_text(strip=True) for span in skills_spans)

        tools_section = description_section.find('strong', text=re.compile(r'Владение инструментами:'))
        if tools_section:
            tools_paragraph = tools_section.find_parent('p')
            if tools_paragraph:
                tools_text = tools_paragraph.get_text(strip=True)
                tools_list = re.sub(r'Владение инструментами:\s*', '', tools_text).replace('Один ', '').strip()

        equipment_section = description_section.find('strong', text=re.compile(r'Снаряжение:'))
        if equipment_section:
            equipment_paragraph = equipment_section.find_parent('p')
            if equipment_paragraph:
                equipment_text = equipment_paragraph.get_text(strip=True)
                equipment_list = re.sub(r'Снаряжение:\s*', '', equipment_text).strip()

    return skills_list, tools_list, equipment_list

def parse_subraces(soup, race_value):
    subraces = []
    subrace_section = soup.find('ul', class_='new-article-menu')
    if subrace_section:
        for subrace_li in subrace_section.find_all('li', class_='new-article-menu__li-second'):
            subrace_link = subrace_li.find('a')
            if subrace_link:
                subrace_value = subrace_link.text.strip()
                subraces.append({'type': 'subrace', 'value': subrace_value, 'parent': race_value})

    return subraces

def parse_data(url, item_type):
    time.sleep(1)  
    response = requests.get(url)
    items = []

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        articles_tiles = soup.find('div', class_='articles-tiles')

        if item_type == 'race':
            ph_tiles = articles_tiles.find_all(class_='tile-PH')
            for tile in ph_tiles:
                race_value = tile.find(class_='article_title').text.strip()
                items.append({'type': 'race', 'value': race_value, 'parent': ''})  
                race_url = tile.find('a')['href'] 
                race_response = requests.get('https://dnd.su' + race_url)
                if race_response.status_code == 200:
                    race_soup = BeautifulSoup(race_response.content, 'html.parser')
                    subraces = parse_subraces(race_soup, race_value)
                    items.extend(subraces)

        elif item_type == 'class':
            for article in articles_tiles.find_all('span', class_='article_title'):
                class_value = article.text.strip()
                items.append({'type': 'class', 'value': class_value, 'parent': ''})

        elif item_type == 'background':
            player_handbook_section = soup.find('div', id='PH')
            for article in player_handbook_section.find_next_siblings('div', class_='col list-item__spell for_filter'):
                background_value = article.find('div', class_='list-item-title').text.strip()
                background_url = 'https://dnd.su' + article.find('a')['href']
                
                background_skills, background_tools, background_equipment = parse_background_skills(background_url)

                items.append({
                    'type': 'background', 
                    'value': background_value, 
                    'parent': '', 
                    'skills': background_skills,
                    'tools': background_tools,
                    'equipment': background_equipment
                })

                if background_value == "Шарлатан":
                    break  # Остановка на "Шарлатане" для того, чтобы не извлекать предыстории, которые в PH не входят

    return pd.DataFrame(items)

df_races = parse_data(url_races, 'race')
df_classes = parse_data(url_classes, 'class')
df_backgrounds = parse_data(url_backgrounds, 'background')

combined_df = pd.concat([df_races, df_classes, df_backgrounds], ignore_index=True)

combined_df.to_csv('data.csv', index=False, encoding='utf-8')


C:\Users\taike\AppData\Local\Temp\ipykernel_45156\1995548772.py:22: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  skills_section = description_section.find('strong', text=re.compile(r'Владение навыками:'))
C:\Users\taike\AppData\Local\Temp\ipykernel_45156\1995548772.py:29: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  tools_section = description_section.find('strong', text=re.compile(r'Владение инструментами:'))
C:\Users\taike\AppData\Local\Temp\ipykernel_45156\1995548772.py:36: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  equipment_section = description_section.find('strong', text=re.compile(r'Снаряжение:'))


Чистим и правим спарсенный csv-файл при помощи регулярок, чтобы создать в нём необходимую для нашего бота структуру:

In [16]:
# Это всё скорее всего можно сделать ещё проще, но это был единственный вариант регулярок, который у меня правильно сработал в функции 
def clean_text(text):
    def clean_within_quotes(match):
        quoted_text = re.sub(r'\s+', ' ', match.group(1)).strip()
        quoted_text = re.sub(r'\bОдин\s+Один\b', 'Один', quoted_text)
        quoted_text = re.sub(r'(?<!\S)(безделушка|безделушки|безделушек)(?!\S)', r' \1 ', quoted_text)
        return '"' + quoted_text + '"'
    
    text = re.sub(r'"(.*?)"', clean_within_quotes, text)
    text = re.sub(r'\b(гном|эльф)\b', '', text)
    text = re.sub(r'\bПрислужник\b', 'Послушник', text) # Чисто косметический момент — мне и моим соигрокам "Послушник" как название этой предыстории просто привычнее
    text = re.sub(r'\s+', ' ', text) 
    text = text.strip()
    text = re.sub(r'\s*,\s*', ', ', text)
    text = re.sub(r'Один\s*видмузыкального', 'Один вид музыкального', text)
    text = re.sub(r'вид\s*музыкального', 'вид музыкального', text)
    text = re.sub(r'одинигровой', 'один игровой', text, flags=re.IGNORECASE)
    text = re.sub(r'вид\s*ремесленных', 'Один вид ремесленных', text) 
    text = re.sub(r'обычной\s*тёмной', 'обычной тёмной', text)
    text = re.sub(r'(?<!\S)(или)(безделушка)', r'\1 \2', text)
    text = re.sub(r'(безделушка|безделушки)(?=\w)', r'\1 ', text) 
    text = re.sub(r'(таблице)(безделушек)', r'\1 \2', text)  

    if text.endswith('.'):
        text = text[:-1] 

    items = [item.strip() for item in text.split(',')]
    seen = set()
    unique_items = []
    for item in items:
        item = re.sub(r'\bОдин\s+Один\b', 'Один', item)
        if item not in seen:
            seen.add(item)
            unique_items.append(item)
    
    text = ', '.join(unique_items)
    
    return text

with open('data.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    header = next(reader)
    cleaned_data = []

    for row in reader:
        if len(row) < 3:
            continue
        
# Убираем/добавляем расы и подрасы, чтобы они соответствовали PH

        is_subrace = row[0] == 'subrace'
        is_race = row[2] in ['Человек', 'Драконорождённый', 'Полуэльф', 'Полуорк', 'Тифлинг']
        contains_latin = bool(re.search(r'[a-zA-Z]', row[1]))
        contains_words = any(word in row[1] for word in ["Разновидности", "Особенности", "Дуэргарские", "Авариэль", "Гругач"])
        contains_defis = '-' in row[1]
        contains_predlogs = 'в' in row[1].split()

        row[1] = clean_text(row[1])

        if not (is_subrace and is_race) and not (contains_latin or contains_words or contains_defis or contains_predlogs):
            cleaned_data.append(row)

final_data = []
added_subraces = set() 

for row in cleaned_data:
    if row[0] == 'race':
        final_data.append(row)
        if row[1] == 'Полуэльф':
            final_data.extend([['subrace', 'Высший', 'Полуэльф'],
                               ['subrace', 'Лесной', 'Полуэльф'],
                               ['subrace', 'Тёмный (дроу)', 'Полуэльф']])
        elif row[1] == 'Тифлинг':
            tifling_subraces = ['Асмодей', 'Мефистофель', 'Зариэль']
            for subrace in tifling_subraces:
                if subrace not in added_subraces:
                    final_data.append(['subrace', subrace, 'Тифлинг'])
                    added_subraces.add(subrace) 
    else:
        final_data.append(row)

# Тут мне захотелось немного поэкспериментировать, потому что таблицу мировоззрений спарсить было неоткуда (не замечала даже раньше, что на русскоязычных днд-ресурсах её нигде нет!)
# Их всего 9 штук, и они комбинируются из параметров морали (добрый, злой, нейтральный) и законности (законный, хаотичный, нейтральный)
moral = ["доброе", "нейтральное", "злое"]
lawfulness = ["законно", "нейтрально", "хаотично"]
alignments = [f'{l}-{m}' for l in lawfulness for m in moral] # Ну я и подумала — а чего бы тогда не покомбинировать их самостоятельно? 

for alignment in alignments:
    if alignment == "нейтрально-нейтральное":
        final_data.append(["alignment", "Истинно-нейтральное"]) # Тоже чисто косметический момент — мне и моим соигрокам такой перевод True Neutral привычнее)) 
    else:
        final_data.append(["alignment", alignment.capitalize()])

with open('data.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(header) 
    writer.writerows(final_data)

df = pd.DataFrame(final_data, columns=['type', 'value', 'parent', 'skills', 'tools', 'equipment'])

for column in df.columns:
    df[column] = df[column].apply(lambda x: clean_text(x) if isinstance(x, str) else x)

df['tools'] = df['tools'].str.capitalize()
df['equipment'] = df['equipment'].str.capitalize()

df.to_csv('data.csv', index=False, encoding='utf-8')

Создаём чат-бота: 

In [17]:
your_bot = "7809129421:AAF76JQEY7hTRfcayXbsHs8L_ttUqkJcybw" 
bot = telebot.TeleBot(your_bot)

def load_data_from_csv(filename):
    data = {
        'races': [],
        'subraces': {},
        'classes': [],
        'backgrounds': [],
        'alignments': [],
        'background_info': {} 
    }

    with open(filename, mode='r', encoding='utf-8') as file: 
        reader = csv.DictReader(file)
        for row in reader:
            key = row['type']
            if key == 'race':
                data['races'].append(row['value'])
            elif key == 'subrace':
                parent = row['parent']
                if parent not in data['subraces']:
                    data['subraces'][parent] = []
                data['subraces'][parent].append(row['value'])
            elif key == 'class':
                data['classes'].append(row['value'])
            elif key == 'background':
                data['backgrounds'].append(row['value'])
                data['background_info'][row['value']] = {
                    'skills': row['skills'],
                    'tools': row['tools'],
                    'equipment': row['equipment']
                }
            elif key == 'alignment':
                data['alignments'].append(row['value'])

    return data

data = load_data_from_csv("data.csv")
races = data['races']
subraces = data['subraces']
classes = data['classes']
backgrounds = data['backgrounds']
alignments = data['alignments']
background_info = data['background_info']

racial_bonuses = {
    "Человек": {"Сила": 1, "Ловкость": 1, "Телосложение": 1, "Интеллект": 1, "Мудрость": 1, "Харизма": 1},
    "Полуэльф": {"Харизма": 2, "Интеллект": 1, "Мудрость": 1},
    "Драконорождённый": {"Сила": 2, "Харизма": 1},
    "Высший": {"Ловкость": 2, "Интеллект": 1},
    "Лесной": {"Ловкость": 2, "Мудрость": 1},
    "Тёмный (дроу)": {"Ловкость": 2, "Харизма": 1},
    "Асмодей": {"Харизма": 2, "Интеллект": 1},
    "Мефистофель": {"Харизма": 2, "Интеллект": 1},
    "Зариэль": {"Харизма": 2, "Сила": 1},
    "Лесной": {"Интеллект": 2, "Ловкость": 1},
    "Скальный": {"Интеллект": 2, "Телосложение": 1},
    "Глубинный (свирфнеблин)": {"Интеллект": 2, "Ловкость": 1},
    "Горный": {"Телосложение": 2, "Сила": 2},
    "Холмовой": {"Телосложение": 2, "Мудрость": 1},
    "Коренастый": {"Ловкость": 2, "Телосложение": 1},
    "Легконогий": {"Ловкость": 2, "Харизма": 1},
}

user_data = {}

def create_inline_keyboard(options):
    markup = types.InlineKeyboardMarkup()
    for option in options:
        markup.add(types.InlineKeyboardButton(option, callback_data=option))
    return markup

def create_start_inline_keyboard():
    markup = types.InlineKeyboardMarkup()
    markup.add(types.InlineKeyboardButton("Создать персонажа", callback_data="create_character"))
    return markup

@bot.message_handler(commands=['start'])
def start(message):
    user_data[message.chat.id] = {
        'race': None,
        'subrace': None,
        'class': None,
        'background': None,
        'alignment': None,
        'abilities': {},
        'points': 24
    }
    bot.send_message(message.chat.id, "Приветствую! Я помощник по созданию персонажа для настольной игры Dungeons & Dragons. Чтобы приступить, нажмите на кнопку ниже. 🎲🐉⚔️", reply_markup=create_start_inline_keyboard())

@bot.callback_query_handler(func=lambda call: call.data == "create_character")
def create_character(call):
    bot.send_message(call.message.chat.id, "Для начала выберите расу вашего персонажа. Она будет определять бонусы к характеристикам и ваше место в лоре Фаэруна.", reply_markup=create_inline_keyboard(races))

@bot.callback_query_handler(func=lambda call: call.data in races)
def choose_race_handler(call):
    user_data[call.message.chat.id]['race'] = call.data
    selected_race = call.data
    
    if selected_race in subraces and subraces[selected_race]:
        bot.send_message(call.message.chat.id, "У выбранной вами расы есть подрасы. Выберите:", 
                         reply_markup=create_inline_keyboard(subraces[selected_race]))
    else:
        bot.send_message(call.message.chat.id, "Отлично! Теперь выберите класс вашего персонажа. Это выбор, который определит ваш стиль игры.", 
                         reply_markup=create_inline_keyboard(classes))

@bot.callback_query_handler(func=lambda call: call.data in [subrace for subs in subraces.values() for subrace in subs])
def choose_subrace(call):
    user_data[call.message.chat.id]['subrace'] = call.data
    selected_subrace = call.data
    bot.send_message(call.message.chat.id, "Отлично! Теперь выберите класс вашего персонажа. Это выбор, который определит ваш стиль игры.", 
                     reply_markup=create_inline_keyboard(classes))

@bot.callback_query_handler(func=lambda call: call.data in classes)
def choose_class(call):
    user_data[call.message.chat.id]['class'] = call.data
    selected_class = call.data
    selected_race = user_data[call.message.chat.id]['race']
    selected_subrace = user_data[call.message.chat.id].get('subrace')

    bot.send_message(call.message.chat.id, "Прекрасно! Теперь выберите предысторию вашего персонажа. Она поможет вам рассказать, как ваш персонаж стал тем, кем является, и предоставит важные сюжетные элементы в его самобытности.", 
                     reply_markup=create_inline_keyboard(backgrounds))

@bot.callback_query_handler(func=lambda call: call.data in backgrounds)
def choose_background(call):
    user_data[call.message.chat.id]['background'] = call.data
    selected_background = call.data

    skills = background_info[selected_background]['skills']
    tools = background_info[selected_background]['tools']
    equipment = background_info[selected_background]['equipment']

    bot.send_message(call.message.chat.id, "Отлично! Теперь выберите мировоззрение вашего персонажа. Этот выбор поможет вам определить его стартовые представления об этике и морали. По мере развития сюжета кампании мировоззрение персонажа может поменяться.", reply_markup=create_inline_keyboard(alignments))

@bot.callback_query_handler(func=lambda call: call.data in alignments)
def choose_alignment(call):
    user_data[call.message.chat.id]['alignment'] = call.data 
    selected_alignment = call.data
    race = user_data[call.message.chat.id].get('race')
    subrace = user_data[call.message.chat.id].get('subrace')
    class_ = user_data[call.message.chat.id].get('class')
    background = user_data[call.message.chat.id].get('background')

    bot.send_message(call.message.chat.id, "Теперь давайте распределим ваши характеристики!\n\nЗначения характеристик помогают определить сильные и слабые стороны вашего персонажа и влияют на бонусы к броскам.\n\nУ вас есть 24 очка для распределения. Пожалуйста, нажмите на кнопку с названием характеристики, которую хотите увеличить.\n\nРуководствуйтесь выбором класса вашего персонажа при распределении. В зависимости от вашего отыгрыша, вы должны иметь две основные характеристики с наибольшим значением. Существует стандартный набор значений для 1-го уровня: 15, 14, 13, 12, 10 и 8, но вы можете поменять его на своё усмотрение. В изначальных значениях учтены бонусы от выбранной вами расы.\n\nИмейте в виду, что значение любой характеристики не должно превышать 20.")

    abilities = {
        "Сила": 8,
        "Ловкость": 8,
        "Телосложение": 8,
        "Интеллект": 8,
        "Мудрость": 8,
        "Харизма": 8
    }
    
    bonuses = racial_bonuses.get(race, {})
    for ability, bonus in bonuses.items():
        if ability in abilities:
            abilities[ability] += bonus

    if subrace:
        subrace_bonuses = racial_bonuses.get(subrace, {})
        for ability, bonus in subrace_bonuses.items():
            if ability in abilities:
                abilities[ability] += bonus

    user_data[call.message.chat.id]['abilities'] = abilities
    user_data[call.message.chat.id]['points'] = 24  

    keyboard = create_inline_keyboard(list(abilities.keys()))

    abilities_message = bot.send_message(call.message.chat.id, 
                                         "Характеристики:\n" + format_abilities(abilities) + 
                                         f"\nОчков для распределения осталось: 24.", 
                                         reply_markup=keyboard)

    user_data[call.message.chat.id]['abilities_message_id'] = abilities_message.message_id

def format_abilities(abilities):
    return '\n'.join([f"{ability}: {value}" for ability, value in abilities.items()])

# Создаём калькулятор модификаторов
def calculate_modifier(value):
    return (value - 10) // 2

@bot.callback_query_handler(func=lambda call: call.data in user_data[call.message.chat.id]['abilities'])
def distribute_points(call):
    if user_data[call.message.chat.id]['points'] > 0:
        selected_ability = call.data
        user_data[call.message.chat.id]['abilities'][selected_ability] += 1
        user_data[call.message.chat.id]['points'] -= 1

        abilities = user_data[call.message.chat.id]['abilities']
        remaining_points = user_data[call.message.chat.id]['points']

        keyboard = create_inline_keyboard(list(abilities.keys()))
        abilities_message = f"Характеристики:\n{format_abilities(abilities)}\nОчков для распределения осталось: {remaining_points}."
        
        bot.edit_message_text(chat_id=call.message.chat.id, message_id=user_data[call.message.chat.id]['abilities_message_id'], text=abilities_message, reply_markup=keyboard)

        if remaining_points == 0:
            race = user_data[call.message.chat.id]['race']
            subrace = user_data[call.message.chat.id].get('subrace')
            class_ = user_data[call.message.chat.id]['class']
            background = user_data[call.message.chat.id]['background']
            selected_alignment = user_data[call.message.chat.id]['alignment']
            abilities = user_data[call.message.chat.id]['abilities']
            skills = background_info[background]['skills']
            tools = background_info[background]['tools']
            equipment = background_info[background]['equipment']

            subrace_display = f"Подраса: {subrace}\n\n" if subrace else ""

            final_message = (f"Раса: {race}\n\n"
                             f"{subrace_display}"
                             f"Класс: {class_}\n\n"
                             f"Предыстория: {background}\n\n"
                             f"Мировоззрение: {selected_alignment}\n\n"
                             f"Характеристики:\n{format_abilities(user_data[call.message.chat.id]['abilities'])}\n\n"
                             f"Модификаторы характеристик:\n" + 
                             "\n".join([f"{ability}: {'+' if calculate_modifier(value) > 0 else ''}{calculate_modifier(value)}" for ability, value in user_data[call.message.chat.id]['abilities'].items()]) + 
                             "\n\n"
                             f"Спасброски: Модификаторы ваших характеристик. Отметьте и увеличьте на 2 те спасброски, модификаторы которых — две ваши характеристикам с наивысшим значением.\n\n"
                             f"Пассивная внимательность: Ваше итоговое значение Мудрости.\n\n"
                             f"Инициатива: Модификатор вашей Ловкости.\n\n"
                             f"Навыки: Модификаторы ваших характеристик. Впишите значения модификаторов в навыки, привязанные к соответствующим характеристикам.\n\n"
                             f"Благодаря выбранной предыстории вы также владеете следующими навыками: {skills}.\nОтметьте их и увеличьте на 2.\n\n"
                     )

            if tools:
                final_message += f"Владения инструментами: {tools}.\n\n"

            final_message += f"Снаряжение: {equipment}.\n\n"

            final_message += (f"Готово! Внесите итоговые значения на лист персонажа в соответствующие ячейки. Не забудьте придумать персонажу имя.\n\n"
                      f"Интересной игры, приключенец! 🧙🏻‍♂️\n\n\n"
                      f"Если хотите создать нового персонажа, напишите /start.")

            bot.send_message(call.message.chat.id, final_message)

if __name__ == "__main__":
    bot.polling(none_stop=True)